In [1]:
# Note: 
# This notebook is very far from the begining, where I was looking into correlations heatmaps and other things.
# Main assumption: isTopPerformer is used by HR to trace valuable students, who produce a lot of not-so-bad code
# Some libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
pd.options.display.max_columns = None

In [2]:
# Data
train = pd.read_csv(f'data/train.csv')
test = pd.read_csv(f'data/test.csv')
course = pd.read_csv(f'data/course.csv')
task = pd.read_csv(f'data/task.csv')
task_results = pd.read_csv(f'data/task_results.csv')

In [3]:
# Just take a look what inside
task.head()

,taskId,createdDate,updatedDate,name,descriptionUrl,description,verification,githubPrRequired,useJury,allowStudentArtefacts,githubRepoName,sourceGithubRepoUrl,type,tags,discipline,attributes,courseTaskId,createdDateCourseTask,updatedDateCourseTask,mentorStartDate,mentorEndDate,maxScore,stageId,scoreWeight,checker,taskOwnerId,studentStartDate,studentEndDate,courseId,pairsCount,typeCourseTask,disabled,createdDateCourse,updatedDateCourse,nameCourse,year,primarySkillId,primarySkillName,locationName,alias,completed,descriptionCourse,descriptionUrlCourse,planned,courseStartDate,courseEndDate,fullName,registrationEndDate,inviteOnly
0,413,2019-08-29 10:57:34.732592,2019-11-11 18:19:01.013044,ST JS Test,http://learn.javascript.ru/,NaN,manual,f,f,f,NaN,NaN,test,NaN,NaN,{},296,2019-08-29 11:01:40.171488,2019-08-29 13:11:10.876319,NaN,NaN,40.0,16.0,1.0,mentor,NaN,2019-04-10 00:00:00+00,2019-04-10 00:00:00+00,12,NaN,NaN,f,2019-08-29 07:26:02.025829,2019-10-25 17:20:30.9813,RS-ST 2019 Q1,NaN,javascript,JavaScript,NaN,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
1,253,2019-05-17 13:02:30.233767,2019-05-17 13:02:30.233767,tdd,NaN,NaN,manual,NaN,f,f,NaN,NaN,NaN,NaN,NaN,{},292,2019-08-29 10:30:11.446774,2019-08-29 13:09:13.674882,NaN,NaN,100.0,16.0,1.0,mentor,NaN,2019-05-24 00:00:00+00,2019-05-24 00:00:00+00,12,NaN,NaN,f,2019-08-29 07:26:02.025829,2019-10-25 17:20:30.9813,RS-ST 2019 Q1,NaN,javascript,JavaScript,NaN,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
2,410,2019-08-29 09:41:00.400898,2019-08-29 10:08:08.993969,ST Chat,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,NaN,NaN,NaN,NaN,NaN,{},288,2019-08-29 09:41:58.974585,2019-08-29 13:09:30.697999,NaN,NaN,120.0,16.0,1.0,mentor,NaN,2019-04-29 00:00:00+00,2019-05-09 00:00:00+00,12,NaN,NaN,f,2019-08-29 07:26:02.025829,2019-10-25 17:20:30.9813,RS-ST 2019 Q1,NaN,javascript,JavaScript,NaN,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
3,407,2019-08-29 09:32:17.606001,2019-08-29 10:08:44.864627,ST Custom Lodash,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,NaN,NaN,NaN,NaN,NaN,{},285,2019-08-29 09:33:10.381953,2019-08-29 13:13:11.434936,NaN,NaN,100.0,16.0,0.8,mentor,NaN,2019-04-15 00:00:00+00,2019-04-21 00:00:00+00,12,NaN,NaN,f,2019-08-29 07:26:02.025829,2019-10-25 17:20:30.9813,RS-ST 2019 Q1,NaN,javascript,JavaScript,NaN,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
4,230,2019-05-17 13:01:38.681206,2019-08-29 10:10:10.985834,ST JS Assignments,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,NaN,NaN,NaN,NaN,NaN,{},289,2019-08-29 09:58:57.892573,2019-08-29 13:12:19.464896,NaN,NaN,150.0,16.0,1.0,mentor,NaN,2019-05-17 00:00:00+00,2019-05-25 00:00:00+00,12,NaN,NaN,f,2019-08-29 07:26:02.025829,2019-10-25 17:20:30.9813,RS-ST 2019 Q1,NaN,javascript,JavaScript,NaN,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t


In [4]:
# Join tasks with task results. 
# From "main assumption" point of view, auto-test tasks should not be taken into account. 
# The same for "presenting" tasks and score for non-coding course activities (aka RS Activist)
# Only useful features are joined for future use
joined_tasks = task_results.set_index(['courseTaskId']).join(task[~task['checker'].isin(
    ['auto-test', 'autoTest'] )& (task['name']!='RS Activist') 
        & (~task['name'].str.contains('resent'))].set_index(
            ['courseTaskId'])[['courseId', 'maxScore', 'scoreWeight', 'name', 'stageId']]).reset_index()

In [5]:
# Join these task results with students from train dataset
# Probably, test and train data should be mixed here...
# Again, only useful features are joined from train for future use
student_tasks = joined_tasks.set_index(['studentId', 'courseId'
            ]).join(train.set_index(
    ['studentId', 'courseId'])[[
    'isTopPerformer', 'mentorId']]).reset_index()

In [6]:
# Here is a map for various expelling reasons. NOT REALLY HELPS 
expelReasonMap = {'Failed stage 1' :0 ,
'no activity': 1,
'no completed tasks':2,
'Course score is too low' :3,
"Your account will be deleted, due to the fact that you did not take an active part in the work on Stage 1 at school iS. Good luck!": 0,
"""Not completed the following tasks in time. 
Virtual Keyboard
Cross-Check: Virtual Keyboard
English for Kids
Cross-Check: English for Kids
Codewars stage 2.
""": 2,
'No activity in private repository': 1,
'Прекратил обучение':4,
'21':10,
'Не вышел на связь': 5,
'Failed Stage 2': 6,
'failed stage#1':0,
'16':32,
'2':22,
'Selected for Short Track 2019 Q3 student': 7,
'200 minimum score criteria is not met':2,
'13':18,
'Too little score': 3,
'17':14,
'Final task score < 40': 3,
"Didn't contact mentor": 5,
'не вышла на связь':5,
'не вышел на связь':5,
"didn't get in touch":5,
'Не вышла на связь': 5,
'1':6,
'18':5,
"Didn't get in touch":5,
"did not pass 2 tasks in a row":7,
"not contacted":5,
"No contact":5,
'miss the deadline': 2,
'9':4,
'11':4,
'По собственному желанию':8,
'Не вышел на связь для интервью в установленный срок.': 5,
'По желанию студента': 8,
"Your account will be deleted, due to the fact that you did not take an active part in the work on Stage 1 at RSSchool iOS. Good luck!":0,
'Not contacted': 5,
"Hasn't contacted for the interview":5 ,
"did not complete 2 tasks in a row": 7,
'no time for studying': 8,
"didn't show up":5,
"didn't contact me": 5,
'At their own request':8,
'Не связался с ментором':5,
'Не связался до 28 числа.':5,
'No contact with me':5,
'не связался':5,
'Студент не вышел на связь':5,
'Did not get in touch.':5,
"decided not to continue to study courses, due to strong employment":8,
'Отказалась продолжать обучение':8,
'failed stage 1':0,
'was no contact for an interview':5,
'refused to continue':8,
"Didn't contact a mentor":5,
"Didn't reach out":5,
'19':2,
'отчислен за списывание': 10,
'Did not contact to set an interview date':5,
'Academic underachievement': 3,
"he hasn't been in touch":5,
"Haven't contacted the mentor (October 28)":5,
"didn't contact":5,
"At the request of the student.":8,
"Your account will be deleted, due to the fact that you did not take an active part in the work on Stage 1 at RSschool iOS. Good luck!":0,
"Отказался от выполнения задания по причине нехватки времени.":8,
"Студент не вышел на связь с ментором для назначения даты интервью в отведенное время":5,
"No actions to contact with mentor":5,
"Нулевой score":3,
"2 tasks":7,
"Didn't contact me":5,
"Didn't contact with me.":5,
"Didn't write to meet":5,
"не сделал 2 таска подряд":7,
"Didn't contact me for interview":5,
"Decided to leave":8,
"отказался дальше учиться":8,
"Not contacting a mentor about an interview.":5,
"Absence of a student":5,
"Didn't pass the Final quiz":11,
"not contacted until October 27":5,
"No tasks were submitted and did not contact the mentor through the course.":2,
"Нету вренми":8,
"Не вышел на связь. Не выполнил два задания.":7,
"Student didn't find me":5,
"did not contact the mentor in time":5,
"Did not contact with me...":5,
"The final quiz hasn't been passed":11,}

In [7]:
# Here is a function which takes threshold for percentage and N for top-N feature and prepares features.
# This approach allowed to tune these threshold level in loop and select the best (helps a bit)
def data_preparation(perc_level, nlarge_level):
    # Just make a copy of previosly prepared dataset
    student_tasks_clr = student_tasks.copy()

    # Fill NA in maxScore with maxScore of the same courseTaskId
    student_tasks_clr['maxScore'] = student_tasks_clr['maxScore'].fillna(
        student_tasks_clr.groupby(['courseTaskId'])['score'].transform('max'))
    
    # Encode isTopPerformer
    student_tasks_clr['isTopPerformer'] = LabelEncoder().fit_transform(student_tasks_clr['isTopPerformer'])

    # Calculate score percentage of maxScore. Probably, using scoreWeight is wrong here, but works better
    student_tasks_clr['perc'] = student_tasks_clr['score'] * student_tasks_clr[
        'scoreWeight']/student_tasks_clr['maxScore']
    
    # Encode mentor presence
    student_tasks_clr['noMentor'] = LabelEncoder().fit_transform(student_tasks_clr['mentorId'].isna())

    # Feature "is student get score higher then maxScore*perc_level"
    student_tasks_clr['hiPerc'] =  LabelEncoder().fit_transform(student_tasks_clr['perc'] > perc_level)
    
    
    # Feature "is student in top-NLARGE_LEVEL by task score". Calculation looks a bit complicated...
    course_student_hi_perc_cnt_1 = student_tasks_clr[student_tasks_clr['perc'] > student_tasks_clr.groupby(
        'courseTaskId')['perc'].nlargest(nlarge_level).min()].groupby(['courseId', 'studentId'])['perc'].count()

    student_tasks_clr = student_tasks_clr.set_index(['courseId', 'studentId']).join(
        course_student_hi_perc_cnt_1.rename('hi_perc_cnt_1')).reset_index()

    # Feature "how many tasks were completed by student in course".
    course_student_tasks_completed = student_tasks_clr[student_tasks_clr['perc'] > 0
                                                      ].groupby(['courseId', 'studentId'])['perc'].count()

    student_tasks_clr = student_tasks_clr.set_index(['courseId', 'studentId']).join(
        course_student_tasks_completed.rename('tasks_completed')).reset_index()

    # "Real" score and maxScore (multipled by scoreWeight)
    student_tasks_clr['adjScore'] = student_tasks_clr['score'] * student_tasks_clr['scoreWeight']
    student_tasks_clr['adjMaxScore'] = student_tasks_clr['maxScore'] * student_tasks_clr['scoreWeight']
    
    
    # Feature "overall student score percentage per course"
    course_student_perc = student_tasks_clr[student_tasks_clr['score'] > 0].groupby(
        ['courseId', 'studentId'])['adjScore'].sum() / student_tasks_clr.groupby(
        ['courseId', 'studentId'])['adjMaxScore'].sum()
    student_tasks_clr = student_tasks_clr.set_index(['courseId', 'studentId']).join(
        course_student_perc.rename('cs_perc')).reset_index()
    
    
    # Feature "average student score percentage for all completed tasks per course"
    course_student_perc_mean = student_tasks_clr.groupby(['courseId', 'studentId'])['perc'].mean()
    student_tasks_clr = student_tasks_clr.set_index(['courseId', 'studentId']).join(
        course_student_perc_mean.rename('perc_mean')).reset_index()
    
    # Return prepared dataframe
    return student_tasks_clr

In [8]:
# Data transfomation function to treat train or test data before fitting
def data_transform(dt, student_tasks_clr):
    # Some encoding
    dt['isExpelledCode'] = LabelEncoder().fit_transform(dt['isExpelled'])
    dt['isFailedCode'] = LabelEncoder().fit_transform(dt['isFailed'])
    
    # Join with prepared dataframe (previous code block)
    dt = dt.set_index(['studentId', 'courseId']).join(
        student_tasks_clr.groupby(['studentId', 'courseId'])[['perc', 'noMentor', 'hi_perc_cnt_1', 
                                                              'hiPerc', 
                                                              'cs_perc', 
                                                              'perc_mean',
                                                              'tasks_completed'
                                                             ]].max()
                                                ).reset_index()
    
    # Student courses count
    dt = dt.set_index('studentId').join(
        student_tasks_clr.groupby('studentId')['studentId'].count().rename('courseCount')).reset_index()
    
    # Expelling reason map and encode
    dt['expellingReasonCode'] = dt['expellingReason'].map(expelReasonMap).fillna(-1)
    
    # NA filling
    dt['courseCount'] = dt['courseCount'].fillna(1)
    dt['noMentor'] = dt['noMentor'].fillna(0)
    dt['perc'] = dt['perc'].fillna(0)
    dt['tasks_completed'] = dt['tasks_completed'].fillna(0)
    dt['hi_perc_cnt_1'] = dt['hi_perc_cnt_1'].fillna(0)
    dt['hiPerc'] = dt['hiPerc'].fillna(0)
    dt['perc_mean'] = dt['perc_mean'].fillna(0)
    dt['cs_perc'] = dt['cs_perc'].fillna(0)
    
    # The function might be used for test dataset, need to check is isTopPerformer exists before encoding
    if 'isTopPerformer' in dt.columns:
        dt['isTopPerformer'] = LabelEncoder().fit_transform(dt['isTopPerformer'])

    # Drop unused features
    dt = dt.drop(columns = ['startDate', 'endDate', 'repository', 'createdDate', 
                            'updatedDate', 'educationHistory', 'repositoryLastActivityDate',
                           'cvUrl', 'hiredById', 'hiredByName', 
                            'preferedMentorGithubId', 'repositoryLastActivityDate',
                           'mentorId', 'readyFullTime', 'englishLevel', 'userId', 'idUser',
                            'courseCompleted',
                            'isExpelled', 'isFailed','expellingReason',
                           ])
    return dt

In [9]:
# Here is "model" tuning: run through percentage level and n-large values, prepare data and train real models
# Results are stored in dataframe for future analysis
some_df = pd.DataFrame(columns=['perc', 'nlarge', 'single', 'courseId'])
for perc_lvl in np.arange(0.6, 0.8, 0.01):
    for nlarge_lvl in np.arange(60, 80, 2):
        row = {'perc': perc_lvl, 'nlarge': nlarge_lvl}
        
        # Prepare and transform data
        curr_df = data_transform(pd.read_csv('data/train.csv'), data_preparation(perc_lvl, nlarge_lvl))
        X_train, X_test, y_train, y_test = train_test_split(
            StandardScaler().fit_transform(
                curr_df.drop(columns=['isTopPerformer', 'studentId'])
            ), curr_df['isTopPerformer'], test_size=0.2)
        
        # Fit model
        model = RandomForestClassifier(random_state=52, criterion='entropy').fit(StandardScaler().fit_transform(
                X_train), y_train)
        # Save resulting score
        row['single'] = f1_score(y_test, model.predict(X_test))

        models={}
        rnd = 52
        total_1 = 0
        total_2 = 0
        
        # Another assumption: courses are independent from each other. 
        # So prepare separate model for each course.
        # Get courseIds
        for course_id in curr_df['courseId'].unique():
            # Filter dataframe
            df2use = curr_df[curr_df['courseId'] == course_id]
            
            X_train, X_test, y_train, y_test = train_test_split(
                StandardScaler().fit_transform(
            df2use.drop(columns=['isTopPerformer', 'studentId', 'courseId'])
            ), df2use['isTopPerformer'],test_size=0.2, random_state=rnd)
            # Fit model
            models[course_id] = RandomForestClassifier(random_state=rnd, criterion='entropy').fit(X_train, y_train)

            # Save "weighted" model score
            total_1 += f1_score(y_test, models[course_id].predict(X_test)) * (len(y_train) + len(y_test))
            total_2 += (len(y_train) + len(y_test))
        
        # Calculate overall score for course models
        row['courseId'] = total_1 / total_2
    # Save modelling results
    some_df = some_df.append(row, ignore_index=True)


In [10]:
# Just sort values in resulting dataframe to get the winner
some_df.sort_values(['single', 'courseId'], ascending=False)

,perc,nlarge,single,courseId
11,0.71,78.0,0.972067,0.965553
1,0.61,78.0,0.969290,0.967525
14,0.74,78.0,0.967054,0.966358
12,0.72,78.0,0.966732,0.965553
15,0.75,78.0,0.966403,0.969055
3,0.63,78.0,0.966394,0.967525
7,0.67,78.0,0.964567,0.968403
19,0.79,78.0,0.964389,0.968103
16,0.76,78.0,0.962547,0.969055
20,0.80,78.0,0.961652,0.967177


In [16]:
# Use the winner. Code is the same as couple blocks before 
nlarge_level = 78
perc_level = 0.71
curr_df = data_transform(pd.read_csv('data/train.csv'), data_preparation(perc_level, nlarge_level))

# Single model fit
X_train, X_test, y_train, y_test = train_test_split(
    StandardScaler().fit_transform(
    curr_df.drop(columns=['isTopPerformer', 'studentId'])
    )
    , curr_df['isTopPerformer'], test_size=0.2)
model = RandomForestClassifier(random_state=52, criterion='entropy').fit(StandardScaler().fit_transform(
        X_train), y_train)
row['single'] = f1_score(y_test, model.predict(X_test))

models={}

rnd = 52
total_1 = 0
total_2 = 0
# Model-per-course fit
for course_id in curr_df['courseId'].unique():
    df2use = curr_df[curr_df['courseId'] == course_id]

    X_train, X_test, y_train, y_test = train_test_split(
        StandardScaler().fit_transform(
    df2use.drop(columns=['isTopPerformer', 'studentId', 'courseId'])
    ), df2use['isTopPerformer'],test_size=0.2, random_state=rnd)
    models[course_id] = RandomForestClassifier(random_state=rnd, criterion='entropy').fit(X_train, y_train)

    total_1 += f1_score(y_test, models[course_id].predict(X_test)) * (len(y_train) + len(y_test))
    total_2 += (len(y_train) + len(y_test))


In [17]:
# Use fitted models to produce output for submission.
test_data = data_transform(pd.read_csv(f'data/test.csv'), data_preparation(perc_level, nlarge_level))
output = pd.DataFrame(columns=['studentId', 'isTopPerformer'])
for course_id in test_data['courseId'].unique():
    tmp_df = test_data[test_data['courseId'] == course_id]
    # If model for courseId were prepared - use it
    if course_id in models:
        y_out = models[course_id].predict(
            StandardScaler().fit_transform(
                tmp_df.drop(columns=['studentId', 'courseId']))
        )     
    # Otherwise - use general model
    else:
        y_out = model.predict(
            StandardScaler().fit_transform(
            tmp_df.drop(columns=['studentId'])
        )
                         )
    output = output.append(pd.DataFrame({'studentId': tmp_df.studentId, 'isTopPerformer': y_out}))

# Convert 1/0 into True/False
output['isTopPerformer'] = output['isTopPerformer'] == 1
# Save output
output.to_csv(f'final_{nlarge_level}_{perc_level}.csv', index=False)